In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from deform_conv import DeformConv2D
from torch.utils.data import DataLoader
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from PIL import Image
from modified_mnist_dataset import ModMNISTDataset

Import our Modified MNIST Dataset. Save this as a .pt file for fast reloading. Only run this if the training.pt and test.pt is not already available.

In [ ]:
import sys
print('[INFO]    Reading training data...')
X = np.loadtxt('./MNIST/comp551w18-modified-mnist/train_x.csv', delimiter=',')
print('[INFO]    Reading training labels...')
Y = np.loadtxt('./MNIST/comp551w18-modified-mnist/train_y.csv', delimiter=',')
X = X.reshape(-1, 64, 64) # reshape 
Y = Y.reshape(-1, 1) 
Image.fromarray(np.uint8(X[100])*255, mode='L')
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.2)
training_set = (torch.from_numpy(X_train), torch.from_numpy(Y_train))
test_set = (torch.from_numpy(X_valid), torch.from_numpy(Y_valid))
print('[INFO]    Writing Torch data files...')
torch.save(training_set, './MNIST/comp551w18-modified-mnist/training.pt')
torch.save(test_set, './MNIST/comp551w18-modified-mnist/test.pt')

In [34]:
#define some args
class Args():  
    batch_size = 32
    test_batch_size = 32
    epochs = 20
    lr = 0.001
    momentum = 0.5
    no_cuda = False
    cuda = False
    seed = 1
    log_interval = 50 
args = Args()

In [35]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

In [4]:
class DeformNet(nn.Module):
    def __init__(self):
        super(DeformNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.offsets = nn.Conv2d(128, 18, kernel_size=3, padding=1)
        self.conv4 = DeformConv2D(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.classifier = nn.Linear(128, 10)

    def forward(self, x):
        # convs
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        # deformable convolution
        offsets = self.offsets(x)
        x = F.relu(self.conv4(x, offsets))
        x = self.bn4(x)

        x = F.avg_pool2d(x, kernel_size=28, stride=1).view(x.size(0), -1)
        x = self.classifier(x)

        return F.log_softmax(x, dim=1)

model = DeformNet()
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))
        if m.bias is not None:
            m.bias.data = torch.FloatTensor(m.bias.shape[0]).zero_()


def init_conv_offset(m):
    nn.init.uniform(m.weight.data)
    if m.bias is not None:
        m.bias.data = torch.FloatTensor(m.bias.shape[0]).zero_()

model = DeformNet()
model.apply(init_weights)
model.offsets.apply(init_conv_offset)
if args.cuda:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

In [37]:
class PlainNet(nn.Module):
    def __init__(self):
        super(PlainNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.classifier = nn.Linear(128, 10)

    def forward(self, x):
        # convs
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        x = F.relu(self.conv4(x))
        x = self.bn4(x)

        x = F.avg_pool2d(x, kernel_size=28, stride=1).view(x.size(0), -1)

        x = self.classifier(x)

        return F.log_softmax(x, dim=1)
    
model = PlainNet()
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))
        if m.bias is not None:
            m.bias.data = torch.FloatTensor(m.bias.shape[0]).zero_()

            model.apply(init_weights)
if args.cuda:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

In [38]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target.squeeze().type(torch.cuda.LongTensor))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target.squeeze().type(torch.cuda.LongTensor), size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.type(torch.cuda.LongTensor).data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [31]:
train_loader = torch.utils.data.DataLoader(
    ModMNISTDataset('./ModMNIST/', train=True,
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    ModMNISTDataset('./ModMNIST/', train=False,
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [ ]:
torch.load('./models/6epoch_checkpoint.pt')

In [39]:
from time import time
for epoch in range(1, args.epochs + 1):
    since = time()
    train(epoch)
    
    iter = time() - since
    print("Spends {}s for each training epoch".format(iter))
    test()

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.325948
Train Epoch: 1 [1600/40000 (4%)]	Loss: 2.304071
Train Epoch: 1 [3200/40000 (8%)]	Loss: 2.271686
Train Epoch: 1 [4800/40000 (12%)]	Loss: 2.297511
Train Epoch: 1 [6400/40000 (16%)]	Loss: 2.298479
Train Epoch: 1 [8000/40000 (20%)]	Loss: 2.294340
Train Epoch: 1 [9600/40000 (24%)]	Loss: 2.308804
Train Epoch: 1 [11200/40000 (28%)]	Loss: 2.301592
Train Epoch: 1 [12800/40000 (32%)]	Loss: 2.287052
Train Epoch: 1 [14400/40000 (36%)]	Loss: 2.293066
Train Epoch: 1 [16000/40000 (40%)]	Loss: 2.316686
Train Epoch: 1 [17600/40000 (44%)]	Loss: 2.291966
Train Epoch: 1 [19200/40000 (48%)]	Loss: 2.300958
Train Epoch: 1 [20800/40000 (52%)]	Loss: 2.268421
Train Epoch: 1 [22400/40000 (56%)]	Loss: 2.281277
Train Epoch: 1 [24000/40000 (60%)]	Loss: 2.255048
Train Epoch: 1 [25600/40000 (64%)]	Loss: 2.298482
Train Epoch: 1 [27200/40000 (68%)]	Loss: 2.302886
Train Epoch: 1 [28800/40000 (72%)]	Loss: 2.260535
Train Epoch: 1 [30400/40000 (76%)]	Loss: 2.299203
Train Epoch: 

Process Process-74:
Traceback (most recent call last):
  File "/home/dias.jason9/anaconda3/envs/py27/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/home/dias.jason9/anaconda3/envs/py27/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dias.jason9/anaconda3/envs/py27/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "modified_mnist_dataset.py", line 58, in __getitem__
    img = Image.fromarray(np.uint8(img.numpy()*255), mode='L')
KeyboardInterrupt


KeyboardInterrupt: 

In [30]:
torch.save(model, './models/20epoch_checkpoint_plainnet.pt')